In [1]:
# constants.py

# 列名
COLUMNS = ['DEPT', 'RMN-RMG', 'CAL', 'SP', 'GR', 'HAC', 'BHC', 'DEN']


In [2]:
# data_read.py

import h5py
import pandas as pd

def load_data_from_h5(filename):
    """
    从.h5文件中读取数据，并返回一个Pandas DataFrame。

    Parameters:
    - filename (str): 文件名，包括路径和扩展名。

    Returns:
    - df (DataFrame): 包含从文件中读取的数据的DataFrame。
    """
    with h5py.File(filename, 'r') as h5f:
        # 获取文件的描述信息
        file_description = h5f.attrs.get('file_description', 'No description')
        print(f"File Description: {file_description}")
        
        # 初始化一个空的DataFrame
        df = pd.DataFrame()
        
        # 遍历文件中的所有数据集
        for key in h5f.keys():
            print(f"Reading {key}...")
            # 读取数据集
            dataset = h5f[key]
            # 将数据集添加到DataFrame中
            df[key] = dataset[:]
            # 打印数据集的单位和描述
            unit = dataset.attrs.get('unit', 'No unit')
            description = dataset.attrs.get('description', 'No description')
            print(f"{key}: Unit = {unit}, Description = {description}")
        
        # 在添加数据集到DataFrame之后，将WellName列中的字节字符串转换为字符串
        df['WellName'] = df['WellName'].apply(lambda x: x.decode('utf-8'))
        return df

filename = './well_log_daqing.h5'
df = load_data_from_h5(filename)
print(df.head())

File Description: This HDF5 file contains oil well log data collected from various wells in the North Sea region. The data includes measurements of depth, resistivity, caliper, sound velocity, gamma ray, and density, among others. The measurements were taken using a combination of logging tools and techniques, including electrical resistivity logging (ERL), caliper logging, and gamma ray logging. The data has been processed to ensure consistency and accuracy, including normalization and interpolation where necessary. This file is intended for use in geological and geophysical analyses, particularly in the study of oil and gas reservoirs. For any questions or further information, please contact the data provider at [email protected]
Reading BHC...
BHC: Unit = us/m, Description = 表面声波时差，没有明确单位，可能是微秒每米 (.us/m)，因为与HAC类似
Reading CAL...
CAL: Unit = cm, Description = 孔隙径，单位为厘米 (.cm)
Reading DEN...
DEN: Unit = g/cm3, Description = 密度，单位为克/立方厘米 (.g/cm3)
Reading DEPT...
DEPT: Unit = m, Descripti

标准化器：

1. standardize
2. maximize

In [3]:
# data_save.py

import h5py
import numpy as np

def save_data_to_h5(data, filename, file_description):
    """
    将数据保存为.h5文件，并添加文件的描述信息作为属性。

    Parameters:
    - data (DataFrame or ndarray): 要保存的数据。
    - filename (str): 文件名，包括路径和扩展名。
    - file_description (str): 整个文件的描述信息。
    """
    with h5py.File(filename, 'w') as h5f:
        # 添加整个文件的描述信息作为根组的属性
        h5f.attrs['file_description'] = file_description
        
        # 创建数据集
        for key in data.columns:
            dataset = h5f.create_dataset(key, data=data[key].values)

In [4]:
import h5py
import numpy as np
import pandas as pd

def load_data_from_h5(filename):
    """
    从.h5文件中读取数据和文件描述信息。

    Parameters:
    - filename (str): 文件名，包括路径和扩展名。

    Returns:
    - data (DataFrame): 从文件中读取的数据。
    - file_description (str): 文件的描述信息。
    """
    with h5py.File(filename, 'r') as h5f:
        # 读取文件描述信息
        file_description = h5f.attrs.get('file_description', 'No description available')

        data = {key: h5f[key][()] for key in h5f.keys()}
        # 将数据转换为DataFrame
        data_df = pd.DataFrame(data)
        
        # 在添加数据集到DataFrame之后，将WellName列中的字节字符串转换为字符串
        data_df['WellName'] = data_df['WellName'].apply(lambda x: x.decode('utf-8'))

        return data_df, file_description



## maximize

In [5]:
from sklearn.preprocessing import MinMaxScaler
preproc = MinMaxScaler()

In [6]:
# data_process.py

import pandas as pd
import joblib
# form constants import COLUMNS

class DataStandardizer:
    """
    数据标准化类，使用提前声明的 preproc 进行标准化。
    """
    def __init__(self, columns=None):
        self.scaler = preproc
        self.columns = columns
        self.fitted = False

    def fit(self, df):
        """
        拟合标准化器。
        """
        if self.columns is None:
            self.columns = df.columns
        self.scaler.fit(df[self.columns])
        self.fitted = True

    def transform(self, df):
        """  
        对数据进行标准化。
        """
        if not self.fitted:
            raise ValueError("Scaler is not fitted yet. Call 'fit' with appropriate data before transforming.")
        # 标准化指定的列
        df_standardized = pd.DataFrame(self.scaler.transform(df[self.columns]), columns=self.columns)
        # 将标准化后的列替换原始数据框中的对应列
        for col in self.columns:
            df[col] = df_standardized[col]
        return df

    def inverse_transform(self, df):
        """ 
        对标准化后的数据进行反标准化。
        """
        if not self.fitted:
            raise ValueError("Scaler is not fitted yet. Call 'fit' with appropriate data before inverse transforming.")
        # 反标准化指定的列
        df_standardized = pd.DataFrame(self.scaler.inverse_transform(df[self.columns]), columns=self.columns)
        # 将反标准化后的列替换原始数据框中的对应列
        for col in self.columns:
            df[col] = df_standardized[col]
        return df

    def save(self, filename):
        """  
        保存标准化器。
        """
        joblib.dump(self.scaler, filename)

    def load(self, filename):
        """ 
        加载标准化器。
        """
        self.scaler = joblib.load(filename)
        self.fitted = True

# 示例使用
standardizer = DataStandardizer(columns= COLUMNS) # 指定需要标准化的列
standardizer.fit(df) # 使用df数据进行标准化
df_MinMaxScaler = standardizer.transform(df) # 标准化df数据

# 保存标准化器
standardizer.save('scaler_minmaxscalr.joblib')

# # 加载标准化器
# standardizer.load('scaler_minmaxscalr.joblib')

# # 反标准化
# df_original = standardizer.inverse_transform(df_MinMaxScaler)

In [7]:
file_description = "将标准化后的数据保存为.h5文件，以便后续使用。标准化器为MinMaxScaler。"
save_data_to_h5(df_MinMaxScaler, 'well_log_daqing_MinMaxScaler.h5', file_description)
print("数据已保存为well_log_daqing_standardized.h5。还有一个标准化器scaler.joblib。使用的标准化器为MinMaxScaler。")

数据已保存为well_log_daqing_standardized.h5。还有一个标准化器scaler.joblib。使用的标准化器为MinMaxScaler。


In [8]:
# 示例使用
filename = './well_log_daqing_MinMaxScaler.h5' # 请替换为您的文件名
data, description = load_data_from_h5(filename)
print("File Description:", description)
print("Data:", data)

File Description: 将标准化后的数据保存为.h5文件，以便后续使用。标准化器为MinMaxScaler。
Data:             BHC       CAL       DEN      DEPT        GR       HAC    RMG  \
0      0.732196  0.583994  0.479137  0.123320  0.011329  0.683669  2.260   
1      0.728385  0.579445  0.486331  0.123416  0.011490  0.668267  2.241   
2      0.725576  0.573612  0.500719  0.123513  0.011468  0.661532  2.405   
3      0.727382  0.568945  0.486331  0.123609  0.011329  0.667949  2.598   
4      0.729586  0.565446  0.500719  0.123706  0.011030  0.689122  2.533   
...         ...       ...       ...       ...       ...       ...    ...   
38729  0.470405  0.307863  0.258993  0.557987  0.006366  0.483154  6.191   
38730  0.461982  0.304363  0.299281  0.558083  0.006727  0.483154  5.867   
38731  0.453761  0.302613  0.328058  0.558179  0.007178  0.482193  6.723   
38732  0.446942  0.307046  0.348201  0.558276  0.007876  0.482193  7.263   
38733  0.426689  0.314512  0.349640  0.558372  0.008394  0.433774  6.874   

         RMN   RMN-R

# 开始训练

In [9]:
import sys
import warnings
import time
from tqdm import tqdm
from math import sqrt

import numpy as np
import pandas as pd

from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

from hyperimpute.plugins.utils.metrics import RMSE
from hyperimpute.plugins.utils.simulate import simulate_nan



import xgboost as xgb

from IPython.display import HTML, display
import tabulate

if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [10]:
from hyperimpute.plugins.imputers import Imputers, ImputerPlugin

imputers = Imputers()
imputers.list()

['most_frequent',
 'sinkhorn',
 'median',
 'sklearn_ice',
 'missforest',
 'miracle',
 'EM',
 'ice',
 'miwae',
 'softimpute',
 'hyperimpute',
 'sklearn_missforest',
 'nop',
 'gain',
 'mice',
 'mean']

In [11]:
custom_ice_plugin = "custom_ice"


class NewPlugin(ImputerPlugin):
    def __init__(self):
        super().__init__()
        lr = LinearRegression()
        self._model = IterativeImputer(
            estimator=lr, max_iter=500, tol=1e-10, imputation_order="roman"
        )

    @staticmethod
    def name():
        return custom_ice_plugin

    @staticmethod
    def hyperparameter_space():
        return []

    def _fit(self, *args, **kwargs) -> "NewPlugin":
        self._model.fit(*args, **kwargs)
        return self

    def _transform(self, *args, **kwargs):
        return self._model.transform(*args, **kwargs)

    def save(self) -> bytes:
        raise NotImplemented("placeholder")

    @classmethod
    def load(cls, buff: bytes) -> "NewPlugin":
        raise NotImplemented("placeholder")


imputers.add(custom_ice_plugin, NewPlugin)

assert imputers.get(custom_ice_plugin) is not None

In [12]:
imputers.list()

['most_frequent',
 'sinkhorn',
 'median',
 'mean',
 'sklearn_ice',
 'missforest',
 'miracle',
 'EM',
 'ice',
 'miwae',
 'softimpute',
 'custom_ice',
 'sklearn_missforest',
 'nop',
 'gain',
 'mice',
 'hyperimpute']

In [16]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

preproc = MinMaxScaler()


def dataset():
    X = data

    COLUMNS = ['DEPT', 'RMN-RMG', 'CAL', 'SP', 'GR', 'HAC', 'BHC', 'DEN']
    
    X = X[COLUMNS]
    
    X = X.head(100)

    return train_test_split(X, test_size=0.2)


def ampute(x, mechanism, p_miss):
    x_simulated = simulate_nan(np.asarray(x), p_miss, mechanism)

    mask = x_simulated["mask"]
    x_miss = x_simulated["X_incomp"]

    return pd.DataFrame(x), pd.DataFrame(x_miss), pd.DataFrame(mask)

['most_frequent',
 'sinkhorn',
 'custom_ice',
 'softimpute',
 'EM',
 'sklearn_missforest',
 'miracle',
 'nop',
 'hyperimpute',
 'mice',
 'sklearn_ice',
 'median',
 'ice',
 'missforest',
 'miwae',
 'mean',
 'gain']

In [17]:
datasets = {}
headers = ["Plugin"]

pct = 0.3

mechanisms = ["MAR", "MNAR", "MCAR"]
percentages = [pct]


plugins = ['most_frequent',
 'sinkhorn',
 'softimpute',
 'EM',
 'sklearn_missforest',
 'miracle',
 'nop',
 'hyperimpute',
 'mice',
 'sklearn_ice',
 'median',
 'ice',
 'missforest',
 'miwae',
 'mean',
 'gain']

X_train, X_test = dataset()

for ampute_mechanism in mechanisms:
    for p_miss in percentages:
        if ampute_mechanism not in datasets:
            datasets[ampute_mechanism] = {}

        headers.append(ampute_mechanism + "-" + str(p_miss))
        datasets[ampute_mechanism][p_miss] = ampute(X_train, ampute_mechanism, p_miss)

In [18]:
import pprint

# 在这里添加打印datasets的代码
for key, value in datasets.items():
    display(f"--- {key} ---") # 使用分隔符和标题
    for sub_key, sub_value in value.items():
        display(f"{sub_key}:")
        display(sub_value)
    display("------") # 使用分隔符

'--- MAR ---'

'0.3:'

(        DEPT   RMN-RMG       CAL        SP        GR       HAC       BHC  \
 53  0.128428  0.296880  0.610126  0.878060  0.008452  0.739490  0.791575   
 12  0.124477  0.300491  0.573378  0.861862  0.009442  0.657361  0.707922   
 96  0.132573  0.299322  0.555180  0.888329  0.008429  0.687519  0.714141   
 57  0.128814  0.295192  0.615959  0.876986  0.008798  0.741415  0.790170   
 55  0.128621  0.296361  0.618525  0.881304  0.008613  0.739172  0.792979   
 ..       ...       ...       ...       ...       ...       ...       ...   
 15  0.124766  0.299010  0.565562  0.862399  0.010777  0.684951  0.738614   
 78  0.130838  0.299322  0.544097  0.881304  0.010547  0.700029  0.723570   
 52  0.128332  0.296932  0.595077  0.877523  0.008314  0.739172  0.789569   
 76  0.130645  0.299296  0.560079  0.878060  0.009005  0.713185  0.730390   
 3   0.123609  0.300569  0.568945  0.864021  0.011329  0.667949  0.727382   
 
          DEN  
 53  0.283453  
 12  0.517986  
 96  0.342446  
 57  0.273

'------'

'--- MNAR ---'

...

'------'

'--- MCAR ---'

'0.3:'

(        DEPT   RMN-RMG       CAL        SP        GR       HAC       BHC  \
 53  0.128428  0.296880  0.610126  0.878060  0.008452  0.739490  0.791575   
 12  0.124477  0.300491  0.573378  0.861862  0.009442  0.657361  0.707922   
 96  0.132573  0.299322  0.555180  0.888329  0.008429  0.687519  0.714141   
 57  0.128814  0.295192  0.615959  0.876986  0.008798  0.741415  0.790170   
 55  0.128621  0.296361  0.618525  0.881304  0.008613  0.739172  0.792979   
 ..       ...       ...       ...       ...       ...       ...       ...   
 15  0.124766  0.299010  0.565562  0.862399  0.010777  0.684951  0.738614   
 78  0.130838  0.299322  0.544097  0.881304  0.010547  0.700029  0.723570   
 52  0.128332  0.296932  0.595077  0.877523  0.008314  0.739172  0.789569   
 76  0.130645  0.299296  0.560079  0.878060  0.009005  0.713185  0.730390   
 3   0.123609  0.300569  0.568945  0.864021  0.011329  0.667949  0.727382   
 
          DEN  
 53  0.283453  
 12  0.517986  
 96  0.342446  
 57  0.273

'------'

In [19]:
from hyperimpute import logger

results = []
duration = []

for plugin in tqdm(plugins):
    plugin_results = [plugin]
    plugin_duration = [plugin]

    for ampute_mechanism in mechanisms:
        for p_miss in percentages:
            print(f"-------Running {plugin} on {ampute_mechanism} with {p_miss} missing values---------")
            # 动态开启debug模式，测试每个模型的训练时间和细节
            logger.add(sink=sys.stderr, level="DEBUG")
            
            ctx = imputers.get(plugin)
            
            x, x_miss, mask = datasets[ampute_mechanism][p_miss]

            start = time.time() * 1000
            x_imp = ctx.fit_transform(x_miss)

            plugin_duration.append(round(time.time() * 1000 - start, 4))
            plugin_results.append(RMSE(x_imp.values, x.values, mask.values))
            print(f"-------{plugin} on {ampute_mechanism} with {p_miss} missing values finished---------")

    results.append(plugin_results)
    duration.append(plugin_duration)

  0%|          | 0/16 [00:00<?, ?it/s][2024-03-18T15:03:02.673424+0800][19916][DEBUG] Loaded plugin imputer - most_frequent
[2024-03-18T15:03:02.714426+0800][19916][DEBUG] Loaded plugin imputer - sinkhorn
[2024-03-18T15:03:02.714426+0800][19916][DEBUG] Loaded plugin imputer - sinkhorn
[2024-03-18T15:03:02.714426+0800][19916][DEBUG] Loaded plugin imputer - sinkhorn
[2024-03-18T15:03:02.714426+0800][19916][DEBUG] Loaded plugin imputer - sinkhorn
[2024-03-18T15:03:02.721429+0800][19916][DEBUG] SinkhornPlugin._fit took 0.0 seconds
[2024-03-18T15:03:02.721429+0800][19916][DEBUG] SinkhornPlugin._fit took 0.0 seconds
[2024-03-18T15:03:02.721429+0800][19916][DEBUG] SinkhornPlugin._fit took 0.0 seconds
[2024-03-18T15:03:02.721429+0800][19916][DEBUG] SinkhornPlugin._fit took 0.0 seconds


-------Running most_frequent on MAR with 0.3 missing values---------
-------most_frequent on MAR with 0.3 missing values finished---------
-------Running most_frequent on MNAR with 0.3 missing values---------
-------most_frequent on MNAR with 0.3 missing values finished---------
-------Running most_frequent on MCAR with 0.3 missing values---------
-------most_frequent on MCAR with 0.3 missing values finished---------
-------Running sinkhorn on MAR with 0.3 missing values---------


[2024-03-18T15:03:11.502677+0800][19916][DEBUG] SinkhornPlugin._transform took 8.7783 seconds
[2024-03-18T15:03:11.502677+0800][19916][DEBUG] SinkhornPlugin._transform took 8.7783 seconds
[2024-03-18T15:03:11.502677+0800][19916][DEBUG] SinkhornPlugin._transform took 8.7783 seconds
[2024-03-18T15:03:11.502677+0800][19916][DEBUG] SinkhornPlugin._transform took 8.7783 seconds
[2024-03-18T15:03:11.511678+0800][19916][DEBUG] SinkhornPlugin._fit took 0.0 seconds
[2024-03-18T15:03:11.511678+0800][19916][DEBUG] SinkhornPlugin._fit took 0.0 seconds
[2024-03-18T15:03:11.511678+0800][19916][DEBUG] SinkhornPlugin._fit took 0.0 seconds
[2024-03-18T15:03:11.511678+0800][19916][DEBUG] SinkhornPlugin._fit took 0.0 seconds
[2024-03-18T15:03:11.511678+0800][19916][DEBUG] SinkhornPlugin._fit took 0.0 seconds


-------sinkhorn on MAR with 0.3 missing values finished---------
-------Running sinkhorn on MNAR with 0.3 missing values---------


[2024-03-18T15:03:19.385246+0800][19916][DEBUG] SinkhornPlugin._transform took 7.8706 seconds
[2024-03-18T15:03:19.385246+0800][19916][DEBUG] SinkhornPlugin._transform took 7.8706 seconds
[2024-03-18T15:03:19.385246+0800][19916][DEBUG] SinkhornPlugin._transform took 7.8706 seconds
[2024-03-18T15:03:19.385246+0800][19916][DEBUG] SinkhornPlugin._transform took 7.8706 seconds
[2024-03-18T15:03:19.385246+0800][19916][DEBUG] SinkhornPlugin._transform took 7.8706 seconds
[2024-03-18T15:03:19.396245+0800][19916][DEBUG] SinkhornPlugin._fit took 0.0 seconds
[2024-03-18T15:03:19.396245+0800][19916][DEBUG] SinkhornPlugin._fit took 0.0 seconds
[2024-03-18T15:03:19.396245+0800][19916][DEBUG] SinkhornPlugin._fit took 0.0 seconds
[2024-03-18T15:03:19.396245+0800][19916][DEBUG] SinkhornPlugin._fit took 0.0 seconds
[2024-03-18T15:03:19.396245+0800][19916][DEBUG] SinkhornPlugin._fit took 0.0 seconds
[2024-03-18T15:03:19.396245+0800][19916][DEBUG] SinkhornPlugin._fit took 0.0 seconds


...

-------gain on MAR with 0.3 missing values finished---------
-------Running gain on MNAR with 0.3 missing values---------


[2024-03-18T15:09:02.089452+0800][19916][DEBUG] GainPlugin._transform took 3.3347 seconds
[2024-03-18T15:09:02.089452+0800][19916][DEBUG] GainPlugin._transform took 3.3347 seconds
[2024-03-18T15:09:02.089452+0800][19916][DEBUG] GainPlugin._transform took 3.3347 seconds
[2024-03-18T15:09:02.089452+0800][19916][DEBUG] GainPlugin._transform took 3.3347 seconds
[2024-03-18T15:09:02.089452+0800][19916][DEBUG] GainPlugin._transform took 3.3347 seconds
[2024-03-18T15:09:02.089452+0800][19916][DEBUG] GainPlugin._transform took 3.3347 seconds
[2024-03-18T15:09:02.089452+0800][19916][DEBUG] GainPlugin._transform took 3.3347 seconds
[2024-03-18T15:09:02.089452+0800][19916][DEBUG] GainPlugin._transform took 3.3347 seconds
[2024-03-18T15:09:02.089452+0800][19916][DEBUG] GainPlugin._transform took 3.3347 seconds
[2024-03-18T15:09:02.089452+0800][19916][DEBUG] GainPlugin._transform took 3.3347 seconds
[2024-03-18T15:09:02.089452+0800][19916][DEBUG] GainPlugin._transform took 3.3347 seconds
[2024-03-1

-------gain on MNAR with 0.3 missing values finished---------
-------Running gain on MCAR with 0.3 missing values---------


[2024-03-18T15:09:05.520400+0800][19916][DEBUG] GainPlugin._transform took 3.3819 seconds
[2024-03-18T15:09:05.520400+0800][19916][DEBUG] GainPlugin._transform took 3.3819 seconds
[2024-03-18T15:09:05.520400+0800][19916][DEBUG] GainPlugin._transform took 3.3819 seconds
[2024-03-18T15:09:05.520400+0800][19916][DEBUG] GainPlugin._transform took 3.3819 seconds
[2024-03-18T15:09:05.520400+0800][19916][DEBUG] GainPlugin._transform took 3.3819 seconds
[2024-03-18T15:09:05.520400+0800][19916][DEBUG] GainPlugin._transform took 3.3819 seconds
[2024-03-18T15:09:05.520400+0800][19916][DEBUG] GainPlugin._transform took 3.3819 seconds
[2024-03-18T15:09:05.520400+0800][19916][DEBUG] GainPlugin._transform took 3.3819 seconds
[2024-03-18T15:09:05.520400+0800][19916][DEBUG] GainPlugin._transform took 3.3819 seconds
[2024-03-18T15:09:05.520400+0800][19916][DEBUG] GainPlugin._transform took 3.3819 seconds
[2024-03-18T15:09:05.520400+0800][19916][DEBUG] GainPlugin._transform took 3.3819 seconds
[2024-03-1

-------gain on MCAR with 0.3 missing values finished---------


In [20]:
display(HTML(tabulate.tabulate(results, headers=headers, tablefmt="html")))
display(HTML(tabulate.tabulate(duration, headers=headers, tablefmt="html")))

Plugin,MAR-0.3,MNAR-0.3,MCAR-0.3
most_frequent,0.0426796,0.0370927,0.0378609
sinkhorn,0.0253901,0.0215528,0.0299175
softimpute,0.0583224,0.036577,0.0321421
EM,0.0208823,0.0224531,0.0256674
sklearn_missforest,0.0172166,0.0182567,0.0191517
miracle,4.30896,4.20978,4.2319
nop,nan,nan,nan
hyperimpute,0.0158683,0.0147342,0.0153443
mice,0.489244,0.0973519,0.16007
sklearn_ice,0.0219315,0.0203346,0.0248199


Plugin,MAR-0.3,MNAR-0.3,MCAR-0.3
most_frequent,3.9961,2.9973,3.999
sinkhorn,8783.25,7877.57,8054.05
softimpute,4806.24,4912.28,4917.34
EM,1000.71,8100.69,6949.41
sklearn_missforest,1611.57,54831,52089.3
miracle,3147.17,4646.81,4923
nop,0,0,1.0024
hyperimpute,20928.2,26940.6,22508.4
mice,1806.52,2473.26,2473.86
sklearn_ice,94.9998,244.56,189.043


毫秒为单位的时间